<h1 align="center">Implementing an autosuggest module</h1> 

---

## Content:

### 1. Why Autosuggest?
### 2. Which Data ?
### 3. Clean Data 
### 4. Which Algorithm? 
### 5. Deploy and integrate..

### 1. Why Autosuggest?
- ⏱️**save your users time**

similar to autocomplete --> suggest most probable keywords. avoid misspellings

Induce a response time constraint ≈ 16ms 
- 💡**provide your user with relevant keywords and/or most frequent queries**

It's very likely that someone else got the exact same problem before you.

- **Optimize an objective** 

An E-commerce website might want to suggest queries which maximize their revenue..

![example](example.png)

### 2. Which Data ?

Ideally logs of user queries.
Often not available when designing and new product.

Alternatives:
1. find Data which looks like user Data.

Webscrapping (search engines, forums, ...)

2. Use Data of your own collections (titles, nouns chunks, keywords extraction...).

You will need a collection of at least a few thousands queries.

⚠️**Queries should  not necessarily match documents of your database**

<h3 align="center">Use of forums posts titles as pseudo queries</h3> 
<img src="forum-screen.png" alt="forum" width="642" align="center">

### Cleaning Data: example with [service-public.fr](https://www.service-public.fr/) Logs

In [144]:
with open("./logs-sp.txt") as f:
    logs = f.read().splitlines()

logs = [l.split(";") for l in logs]
logs[:10] # couples (query text, number of occurences)

[['filtre', '508627'],
 ['etatcivil', '381965'],
 ['filtre', '295580'],
 ['jechangedecoordonnees', '236041'],
 ['inscriptionelectorale', '179999'],
 ['acte de naissance', '79663'],
 ['fcb', '43800'],
 ['insregistrefr', '32524'],
 ['md', '31369'],
 ["carte d'identité", '21107']]

In [155]:
logs = logs[5:] # remove first 5 queries
print(len(logs)) # number of unique requests
print(sum([int(l[1]) for l in logs])) # number of aggregated requests

199966
2439050


Cleaning optionnally involves:
    - lowercase
    - remove some punctuation
    - remove noise
    - remove accents with ascii
    - remove long digits
    - remove dates ...
    - remove user data..
    - remove uninformative queries ("help", "jechangedecoordonnees")
    - filter out queries having occurence under T
    - remove queries with small edit distance to frequent queries (carte d'identité, carte d identité)
    - remove misspelled queries (usually around 20%) 

- most of the time, the cleaning phase is dataset dependent

### 4. Which Algorithm ? 

-  suggestion time should remain below 50ms 
    - Keep Complexity low
    
 
- your query/document list probabbly will evolve/grow with time



- the algorithm has 2 steps:
    - indexing step (does'nt need to be fast)
    - search step (keep as low as possible)

its easy to trade memory against speed at indexing time

# Simple baseline

In [175]:
# Simple baseline:

# at search time: sort request by frequencies
# filter out the documents 

prefix = "cong"
topn = 15

[l for l in logs if l[0].startswith(prefix)][:topn]

[['congé parental', '2742'],
 ['congé maternité', '1598'],
 ['congé paternité', '912'],
 ['congés payés', '904'],
 ['congé de paternité', '640'],
 ['congé sabbatique', '497'],
 ['congés annuels', '488'],
 ["congé parental d'éducation", '481'],
 ['congé de naissance', '448'],
 ['congés', '425'],
 ['conges payes', '328'],
 ['congé', '328'],
 ['conge parental', '308'],
 ['congé payé', '306'],
 ['congé sans solde', '288']]

# simulate with ipywidgets

In [185]:
from ipywidgets import interact

def search(prefix):
    return [l for l in logs if l[0].startswith(prefix)][:topn]

inter = interact(search, prefix = "")

A Jupyter Widget

# Complexity of O(N), with N: nb of queries

In [168]:
%timeit [l for l in logs if l[0].startswith(prefix)][:topn]

37.6 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [198]:
# lets make the logs list 10 times bigger:

big_logs = logs * 10
big_logs = sorted(big_logs, key=lambda x: int(x[1]), reverse=True)
len(big_logs)

1999660

In [197]:
%timeit [l for l in big_logs if l[0].startswith(prefix)][:topn]

353 ms ± 7.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Must read:

* [a great blog post about autosuggest optimization and data structures](https://medium.com/related-works-inc/autosuggest-retrieval-data-structures-algorithms-3a902c74ffc8)

# Trade Memory for cpu time

In [232]:
# Idea: make a hashtable of queries by starting string 
# search only in this table 
class Search():
    def __init__(self, logs):
        self.index(logs)
    
    def index(self, logs):
        self.first_char = set([l[0][0] for l in logs]) # find all starting characters
        # make an index with all first char

        self.hashtab = {}
        for char in self.first_char:
            self.hashtab[char] = [l for l in logs if l[0].startswith(char)]
            
    def search(self, prefix):
        subset = self.hashtab[prefix[0]]
        return [l for l in subset if l[0].startswith(prefix)]

In [239]:
s = Search(logs) # build hashtable

In [241]:
%timeit s.search(prefix) # perform actual search

11.6 ms ± 323 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### You want fancier algorithm?

    - handle typos with fuzzy search
    - tree search
    - seasonality 
    - Include User preferences
    - Learning to rank
    - NLP models (LM, clustering, Neural Query Embeddings etc..). 

### blog post from etsy: 
- [part 1 Data Scturctures and Optimization](https://medium.com/related-works-inc/autosuggest-retrieval-data-structures-algorithms-3a902c74ffc8)
- [part 2: NLP and Fancy algorithms](https://medium.com/related-works-inc/autosuggest-ranking-d8a3242c2837)

# Deployment and integration with Elastic:

- For each search ≈2 suggestions, it's better to have a microservice.
- if your application is overloaded, the autosuggest will probably be down first.


we deployed as a [Flask/Gunicorn API](https://github.com/SocialGouv/code-du-travail-numerique/tree/master/packages/code-du-travail-nlp/api): (a simple GET route):
- get prefix -> return an array of suggestions. 

- wrapped with your front ( few [lines of javascript](https://github.com/SocialGouv/code-du-travail-numerique/blob/master/packages/code-du-travail-frontend/src/common/Suggester.js) with debunking to avoid flooding)

NOTE: Elastic has a [built-in suggester](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-suggesters-completion.html) based on Indexed documents (search as you type)
